In [1]:
!pip install -qU langchain-ollama langchain_chroma langchain
!pip install -qU ollama

^C
ERROR: Operation cancelled by user


In [2]:
from typing import List
import re

from ollama import chat
from ollama import ChatResponse
from langchain_ollama import OllamaEmbeddings
from IPython.display import display, Markdown, Latex
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter


### База данных
В качетсве базы данных используем хатеханный госбук по ГОСу по математике 2022г. Цель состоит в том, чтобы модель давала релевантные ответы по соновным вопросам в области математики

### Создание новой векторную базу данных

Относительно долгиий процесс, можно использовать готовую базу, сохраненную на гитхабе

In [3]:
num_chapters = 38
chapters_pages = []
for i in range(1, num_chapters + 1):
    text = ""
    with open(f'./chapters/chapter{i}.tex', 'r') as f:
        text = f.read()
    chapters_pages.append(text)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=16384, 
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.create_documents(chapters_pages)

В качестве модели для эмбедингов используем mxbai-embed-large из ollama. Она создает наиболее релевантные эмбединги текстов для запросов по сравнению с сотальными моделями из ollama.

In [4]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

В качестве векторной базы данных используется ChromaDB. Также были эксперименты в ноутбуке dev с faiss, но ChromaDB была выбрана т.к. имеет более простой и удобочиитаемый pipeline.

In [5]:
vector_store = Chroma.from_documents(
    chunks,
    embeddings,
    collection_name="gosbook_collection",
    persist_directory="./gosbook_collection_db",  # Where to save data locally, remove if not necessary
)

KeyboardInterrupt: 

### Использовать готовую сохраненную базу данных

Уже есть готовая сохраненная база данных по госбуку, поэтому можно исполнить просто код ниже

In [6]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [7]:
vector_store = Chroma(
    collection_name="gosbook_collection",
    embedding_function = embeddings,
    persist_directory="./gosbook_collection_db",  # Where to save data locally, remove if not necessary
)

## Выделить нужные фрагменты тескста

Если в базе данных искать вопрос напрямую, то не всегда выдаются релевантные части текста. Это связано с тем, что большая часть математических текстов лишина семантики, т.к. состоит из формул. Поэтому предлагается искать по правдоподобным ответам по запросу.

In [8]:
def guess_answer_phi(question, model="phi4"):
    user_message = f"""

    Придумай правдоподобный ответ на вопрос.
    Query: {question}
    Answer: 
    """
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    response: ChatResponse = chat(model=model, messages=messages)
    answer = f"""
    {question}
    ---------------------
    {response.message.content}
    """
    return answer

А вот уже к составленному промпту можно искать наиболее близкиие тексты.

In [9]:
def retrive_chunks_for_question(question: str, vector_store: Chroma) -> List[str]:
    prompt_question = guess_answer_phi(question)
    retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
    )
    res =  retriever.invoke(prompt_question)
    retrieved_chunk = [e.page_content for e in res]
    return retrieved_chunk

В промти просто загоняем полученный контекст и изначальный вопрос. Также добавлем некторые пожелания по запросу

In [10]:
def get_prompt_from_answer(retrieved_chunk: List[str], question:str) -> str:
    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Answer in Russian. Замени в ответе все символы '\\(' и '\\)' на символ '$'.
    Query: {question}
    Answer:
    """
    return prompt


В качестве модели был выбран Phi4, она хорошо справлется с запросами на русском языке и математике. 

In [11]:
def run_phi(user_message: str, model: str="phi4") -> str:
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    response: ChatResponse = chat(model=model, messages=messages)
    return response.message.content

Ответы приходится немного отформатировать, чтобы Latex заменить на MathJax

In [12]:
def format_to_mathjax(unformated_text: str) -> str:
    pattern = r"(\\\()|(\\\))"
    repl = '$'
    result_, _ = re.subn(pattern, repl, unformated_text)
    pattern = r"\\\[|\\\]"
    repl = '$$'
    formated_text, _ = re.subn(pattern, repl, result_)
    return formated_text

### Готовый pipeline для ответа на вопрос с помощью ГосБука для выпускного экзамена по математике

Все фуннкции выше обюъединим в одну готовую

In [13]:
def get_answer_from_gosbook_database(question: str, gb_database: Chroma):
    retrieved_chunk: List[str] = retrive_chunks_for_question(question, gb_database)
    question_prompt: str = get_prompt_from_answer(retrieved_chunk, question)
    unformatted_answer: str = run_phi(question_prompt)
    answer: str = format_to_mathjax(unformatted_answer)
    return answer

## Примерный пак вопросов

In [23]:
question = "Какие особенности у формулы Тейлора с остаточным членом в форме Пеано?"

In [ ]:
question = "Как исследовать функцию на минимум/максимум?"

In [31]:
question = "Вид формулы Тейлора с остаточным членом в форме Пеано. И пример с применение этой формулы"

In [ ]:
question = "Критерий Коши сходимости числовой последовательности"

In [ ]:
question = "Критерий Коши для функционыльных рядов"

In [18]:
question = "Вид формулы Тейлора с остаточным членом в форме Пеано. И пример с применение этой формулы для функции f(x) = log(1 + exp(x)). Ответь максимально коротко"

In [19]:
answer = get_answer_from_gosbook_database(question, vector_store)

KeyboardInterrupt: 

In [33]:
print(question)

Вид формулы Тейлора с остаточным членом в форме Пеано. И пример с применение этой формулы


In [34]:
display(Markdown(answer))

Формула Тейлора с остаточным членом в форме Пеано выглядит следующим образом:

$$ f(x) = P_n(x) + R_n(x), $$

где $ P_n(x) $ — многочлен степени $ n $, который представляет собой разложение функции $ f $ в ряд Тейлора до $ n $-го члена, а $ R_n(x) $ — остаточный член, определяемый как:

$$ R_n(x) = o((x-a)^n), $$

где $ a $ — точка разложения. Это значит, что

$$
\lim_{x \to a} \frac{R_n(x)}{(x-a)^n} = 0.
$$

**Пример применения:**

Рассмотрим функцию $ f(x) = e^x $, которую хотелось бы разложить в ряд Тейлора около точки $ a = 0 $.

Многочлен степени $ n $ для этой функции имеет вид:

$$ P_n(x) = 1 + x + \frac{x^2}{2!} + \cdots + \frac{x^n}{n!}. $$

Остаточный член в форме Пеано будет

$$ R_n(x) = o(x^n). $$

Таким образом, разложение функции $ f(x) = e^x $ с остаточным членом в форме Пеано около точки $ 0 $ имеет вид:

$$ e^x = 1 + x + \frac{x^2}{2!} + \cdots + \frac{x^n}{n!} + o(x^n). $$

Это означает, что при приближении к нулю разность между функцией $ e^x $ и её $ n $-м степенным приближением делится на любую положительную величину больше $ x^n $, что делает эту разность бесконечно малой относительно $ x^n $.